In [1]:
import pandas as pd
import numpy as np
import duckdb
import os
import sys
from utils import time, data_processor, constants

In [2]:
all_countries = set(constants.eu + constants.americas + constants.asia + constants.africa + constants.oceania)
eu = set(constants.eu)
balkans = set(constants.balkans)

In [ ]:
balkans

In [8]:
INDUSTRY_CLASSIFICATION_PATH = "../../data_raw/firmographics/industry_classifications/*.parquet"
CONTACT_INFO_PATH = "../../data_raw/firmographics/contact_info/*.parquet"
LEGAL_INFO_PATTERN = "../../data_raw/firmographics/legal_info/*.parquet"

In [9]:
# def fetch_data(country_iso_code):
#     query = f"""
#     SELECT
#         *
#     FROM
#         '{CONTACT_INFO_PATH}' as nace
#     WHERE
#         nace."bvd_id_number" LIKE '{country_iso_code}%'

#     """
#     conn = duckdb.connect()
#     df = conn.execute(query).fetchdf()
    
#     return df

# # ita_industry_classification = fetch_data("IT")
# ita_contact_info = fetch_data("IT")
# # ita_legal_info = fetch_data("IT")

## NON-HISTORICAL DATA

In [10]:
def fetch_data_for_country(country_iso_code):
    query = f"""
        SELECT 
            nace.bvd_id_number,
            nace.nace_rev_2_core_code_4_digits_,
            contact.nuts2,
            contact.city_native_,
            legal_info.status,
            legal_info.type_of_entity
        FROM
            '{INDUSTRY_CLASSIFICATION_PATH}' AS nace
        LEFT JOIN
            '{CONTACT_INFO_PATH}' AS contact
        ON
            nace.bvd_id_number = contact.bvd_id_number
        LEFT JOIN
            '{LEGAL_INFO_PATTERN}' AS legal_info
        ON
            nace.bvd_id_number = legal_info.bvd_id_number
        WHERE
            nace.bvd_id_number IS NOT NULL
        AND 
            (nace.nace_rev_2_core_code_4_digits_ IS NOT NULL 
            OR contact.nuts2 IS NOT NULL 
            OR legal_info.status IS NOT NULL 
            OR legal_info.type_of_entity IS NOT NULL)
        AND
            contact.country_iso_code = '{country_iso_code}'
    """

    conn = duckdb.connect()
    df = conn.execute(query).fetchdf()
    
    return df



In [11]:
country = 'KV' 
year = 2018

kv =  fetch_data_for_country(country)

In [ ]:
output_path = "..\\..\\data_processed\\firmographics_processed\\"

for country in balkans:
    df = fetch_data_for_country(country)
    df.to_parquet(f"{output_path}firmographics_{country}.parquet")
    print(f"{country}")

In [14]:
# TEMP_TABLE_KEY_FINANCIALS = "..\\..\\data_raw\\key_financials\\key_financials_eur\\*.parquet"

# def fetch_data_for_country_historical(country_iso_code):
#     query = f"""
#     SELECT 
#         number_of_employees, 
#         closing_date,
#         bvd_id_number,
#         total_assets,
#         operating_revenue_turnover_,
#         EXTRACT(YEAR FROM closing_date) AS year  
#     FROM 
#         '{TEMP_TABLE_KEY_FINANCIALS}' 
#     WHERE
#         bvd_id_number LIKE '{country_iso_code}%'
#         """
    
#     # Connect to DuckDB and execute the query   
#     conn = duckdb.connect()
#     df = conn.execute(query).fetchdf()
#     return df

In [ ]:
# ## Test
# # country = "IT"
# OUTPUT_PATH = "..\\..\\data_processed\\key_financials_processed\\"

# for country in all_countries:
#     df = fetch_data_for_country_historical(country)
#     os.makedirs(OUTPUT_PATH, exist_ok=True)
#     df.to_parquet(f"{OUTPUT_PATH}\\key_financials_processed_{country}.parquet")
#     print(f"{country}")


In [ ]:
cs = pd.read_parquet(f"{output_path}firmographics_CS.parquet")
cs